In [6]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import csv
import random

acc_login = "Login or username here"  # your login here
acc_password = "Password here"  # your pass here

# GIFs to collect the Xpaths for each of the functions is included in the GitHub

wait = random.gauss(10, 5) # Adding 3 different random gaussian/normal distributions for the different periods 
wait2 = random.gauss(15, 3)
wait3 = random.gauss(17, 3)
def scrolling(url, scrolling_on=False, comments=False, see_more=False):
    chrome_path= "/Users/gowthamasokan/Documents/RestaurantWaitTime/chromedriver"

    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized");
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")
    driver = webdriver.Chrome(executable_path=chrome_path, options=options)

    driver.get(url)

    time.sleep(wait)

    username = driver.find_element_by_id("email")
    password = driver.find_element_by_id("pass")
    submit = driver.find_element_by_id("loginbutton")

    username.send_keys(acc_login)  # you can set your log
    password.send_keys(acc_password)  # and pass in set_settings
    submit.click()

    # browser = scrollDown(driver, 20)
    time.sleep(wait2)

    # scroll_down(driver)

    if scrolling_on:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(wait3)

    if see_more:
        try:
            more = driver.find_elements_by_xpath(
                '/html/body/div[1]/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[3]/div[6]/div[4]/div/div[1]/div[74]/div/div[2]/div[1]/div[3]/div[3]/div[2]/div/div/div[2]/div/span/span/a')[
                0]
            more.click()
            time.sleep(5)
        except:
            pass

    if comments:

        view_previous_comments = driver.find_elements_by_xpath(
            '/html/body/div[1]/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[3]/div[6]/div[4]/div/div[1]/div[81]/div/div[2]/div[2]/form/div/div[3]/div[1]/div/a')
        while len(view_previous_comments) != 0:
            view_previous_comments[0].click()
            time.sleep(2)
            view_previous_comments = driver.find_elements_by_xpath(
                '/html/body/div[1]/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[3]/div[6]/div[4]/div/div[1]/div[81]/div/div[2]/div[2]/form/div/div[3]/div[1]/div/a')

        more_comments = driver.find_elements_by_xpath(
            '/html/body/div[1]/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[3]/div[6]/div[4]/div/div[1]/div[8]/div/div[2]/div[2]/form/div/div[3]/div[1]/div')
        while len(more_comments) != 0:
            more_comments[0].click()
            time.sleep(5)
            more_comments = driver.find_elements_by_xpath(
                '/html/body/div[1]/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[3]/div[6]/div[4]/div/div[1]/div[8]/div/div[2]/div[2]/form/div/div[3]/div[1]/div')

        replies = driver.find_elements_by_xpath(
            '/html/body/div[1]/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[3]/div[6]/div[4]/div/div[1]/div[19]/div/div[2]/div[2]/form/div/div[3]/ul/li[1]/div[2]/div/a')
        for r in replies:
            r.click()
            time.sleep(2)

        see_more_comments = driver.find_elements_by_xpath(
            '/html/body/div[1]/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[3]/div[6]/div[4]/div/div[1]/div[78]/div/div[2]/div[2]/form/div/div[3]/ul/li[6]/div[1]/div/div[2]/div/div[1]/div[1]/div/div/div/span/span/a')
        for s in see_more_comments:
            s.click()
            time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    driver.close()
    return soup




soup = scrolling('https://www.facebook.com/groups/reopenusa2020', scrolling_on = True)

post_links = soup.find_all(class_ = '_5pcq')

permalinks = []
for link in post_links:
    cleaned_link = link['href']
    try:
        permalink = cleaned_link.split('/')[4] # Need to change the index depending on how far the permalink number is from the first slash counted by the following slashes
        permalinks.append(permalink)
    except:
        pass

permalinks = pd.DataFrame(permalinks)

permalinks.to_csv('./permalinks.csv', header = False, index = False)

permalinks = pd.read_csv('./permalinks.csv', header = None)

with open('./posts.csv', "w", newline='') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(['Permalink', 'Post'])  # write the header

with open('./comments.csv', "w", newline='') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(['Permalink', 'Comment'])  # write the header

for i in range(len(permalinks)):
    posts = []
    all_comments = []
    let = str(permalinks.loc[i, :].values[0]) # Converting the permalinks to strings
    let = let[:-2] # Slicing the last two .0 characters from the string
    print(let)
    print(type(let))
    url = 'https://www.facebook.com/groups/reopenusa2020/permalink/' + let + '/'
    post_soup = scrolling(url, comments=True, see_more=True)
    try:
        posts_single = {}
        post = post_soup.find(
            class_='_5pbx userContent _3576').text
        timestamp = post_soup.find(class_='timestampContent').text
        posts_single['Permalink'] = let
        posts_single['post'] = post
        posts_single['timestamp'] = timestamp
        posts.append(posts_single)
        posts_df = pd.DataFrame(posts)
        print(posts_df)
        posts_df.to_csv('./posts.csv', mode='a', index=False, header=False)
    except:
        pass

    try:
        comments = post_soup.find_all(class_='_3l3x')
        for c in comments:
            comments_post = {}
            comments_post['Permalink'] = let
            comments_post['Comment'] = c.text
            all_comments.append(comments_post)
        all_comments_df = pd.DataFrame(all_comments)
        print(all_comments_df)
        all_comments_df.to_csv('./comments.csv', mode='a', index=False, header=False)
    except:
        pass



1456237437890333
<class 'str'>
          Permalink                                               post  \
0  1456237437890333  Good afternoon Patriots! We have big plans for...   

            timestamp  
0  May 20 at 12:43 PM  
           Permalink                                            Comment
0   1456237437890333                                          Thank you
1   1456237437890333  Shouldn't we have backup on MeWe in case Faceb...
2   1456237437890333  See this https://www.thegatewaypundit.com/.../...
3   1456237437890333  Re-Open AmericaMother Of All Bombs! (start wat...
4   1456237437890333   Why did one person hit the laugh button on this?
5   1456237437890333     Thank you , Craig  Derry !  So vital to know !
6   1456237437890333   Why did Alegandro Roberto laugh at this?  Admin?
7   1456237437890333  Thank you for the invite.  Have a Great Memori...
8   1456237437890333  What this silly meme fails to mention is almos...
9   1456237437890333  Craig Dery I have legal, actio

          Permalink                                               post  \
0  1464372097076867  https://facebook.com/events/s/we-are-washingto...   

           timestamp  
0  May 29 at 4:36 PM  
          Permalink                                            Comment
0  1464372097076867               This needs to be done in every state
1  1464372097076867  It's going on every where yesterday across sta...
2  1464372097076867  Greetings from Texas. i want to give thanks to...
1464325527081524
<class 'str'>
          Permalink                                               post  \
0  1464325527081524  Looking forward to seeing you all that can mak...   

           timestamp  
0  May 29 at 4:47 PM  
          Permalink           Comment
0  1464325527081524  Wish I was there
1468954873285256
<class 'str'>
           Permalink                                            Comment
0   1468954873285256                                 Where's the Bible?
1   1468954873285256  If that’s a real bible

           Permalink                                            Comment
0   1465751750272235                                  This is old video
1   1465751750272235  Those are police... not military. Thats a humb...
2   1465751750272235  Also..... shoot them back. Woth real bullets. ...
3   1465751750272235    Didn’t this video circulate a couple months ago
4   1465751750272235  Shaun King, huh?  The stupid mother fucker who...
5   1465751750272235  Report this false video to admins, it don't be...
6   1465751750272235             The comments alone made it worth it.🤣😂
7   1465751750272235                      That’s not a tank you dumbass
8   1465751750272235                 Shaun king is a poser and wanna be
9   1465751750272235  Cowards hiding behind a mask! If you believe y...
10  1465751750272235  I don't agree that these people at home should...
11  1465751750272235                          Take this fake shit down.
12  1465751750272235  So we’re going to bitch about these “anima

KeyboardInterrupt: 